**Cepstrum**: a waveform that went through inverse fast fourier transform from spectrum. 
    **aims to extract formant** elements, which is in lower range of frequencies. Artiulcated phones are distinguished using this information. 


- required packages
    - wave
    - numpy
- variables
    - wav_path: the wave file that you want to cepstrum analyze
    - target_time: the time at which the target phone starts
    - fft_size: the number of samples to conduct fast fourier transform
    - cep_threshold: the boundary dimension to decide the low and high frequencies
        
- steps
    - open wave file
    - proceed fft to create spectrum and make it log-power
    - proceed ifft to create cepstrum
        - extract cepstrum_low 
        - run fft to create log_power_ceplo
        - extract cepstrum_high
        - rin fft to create log_power_cephigh




In [ ]:
import wave
import numpy as np
import matplotlib.pyplot as plt

if __name__ == '__main__':
    wav_path = '# target file wave path from which to create cepstrum'
    target_time = 0.32 # target time when the target phone starts at
    fft_size = 1024 # the number of samples to use as unit to run fast fourier trasnform
    cep_threshold = 33 # the dimension with which to decided the low and high frequency range

    with wave.open(wav_path) as wav:
        sampling_frequency = wav.getframerate()
        num_samples = wav.getnframes()
        waveform = wav.readframes(num_samples)
        waveform = np.frombuffer(waveform)

    # create target index using target time[s] * sampling frequency[the number of samples / total recorded time]
    target_index = int(target_time * sampling_frequency)
    # extract target frame
    frame = waveform[target_index:(target_index + fft_size)].copy()
    # proceed fast fourier transform
    spectrum = np.fft.fft(frame)
    # convert as log-powered
    log_power = np.power(np.log(np.abs(spectrum) + 1E-7), 2)

    # proceed inverse fast fouriere transfomation 
    cepstrum = np.fft.ifft(log_power)
    # extract cepstrum of low frequencies
    cepstrum_low = cepstrum.copy()
    cepstrum_low[cep_threshold+1:-cep_threshold] = 0
    # create log powered cepstrum of low frequencies i.e. formants
    log_power_ceplo = np.abs(np.fft.fft(cepstrum_low))
    # extract cepstrum of high frequencies
    cepstrum_high = cepstrum - cepstrum_low
    cepstrum_high[0] = cepstrum[0]
    # create log powered cepstrum of high frequencies
    log_power_cephi = np.abs(np.fft.fft(cepstrum_high))

    # create axis of frequencies to plot log_power spectrum
    freq_axis = np.arange(fft_size) * sampling_frequency / fft_size
    # create axis of quefrencies to plot cepstrum
    qefr_axis = np.arange(fft_size) / (fft_size * sampling_frequency)

    cepstrums = [cepstrum, cepstrum_low, cepstrum_high]
    log_powers = [log_power, log_power_ceplo, log_power_cephi]

    for n, log_pow in enumerate([log_power, 
                                log_power_ceplo, 
                                log_power_cephi]):
        plt.subplot(3, 2*n + 1)
        plt.plot(freq_axis, log_pow, color = 'k')
        plt.xlabel('Freq [Hz]')
        plt.ylabel('Value')
        plt.xlim([0, fft_size / 2])
        plt.ylim([0, 30])
    
    for n, cep in enumerate([cepstrum, 
                            cepstrum_low, 
                            cepstrum_high]):
        plt.subplot(3, 2, 2*n + 2)
        plt.plot(qefr_axis, np.real(cep), color = 'k')
        plt.xlabel('Time [sec]')
        plt.ylabel('Value')
        plt.xlim([0, fft_size / (sampling_frequency * 2)])
        plt.ylim([0, 30])